<h2 style='color:Blue'> Model Development

***Implementtaion of published paper***

<h4 style='color:brown'> Claimed Unique Findings of Each Paper </h4>

| **Paper Title**                                                                 | **Unique Findings**                                                                                           |
|---------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------|
| **An Extended Machine Learning Technique for PCOS Detection Using Ovary Ultrasound Image** | - Combines CNN with XGBoost for an ensemble model achieving 99.89% accuracy.                                 |
|                                                                                 | - Focuses on transfer learning and feature reduction for computational efficiency.                            |
|                                                                                 | - Introduces a stacking model outperforming traditional ML and standalone deep learning techniques.           |
| **CystNet: An AI Driven Model for PCOS Detection Using Multilevel Thresholding** | - Proposes CystNet, a hybrid model integrating InceptionNet V3 with autoencoders for feature extraction.      |
|                                                                                 | - Implements Watershed and multilevel thresholding techniques for advanced segmentation.                      |
|                                                                                 | - Achieves 97.75% accuracy through ensemble classifiers and rigorous evaluation metrics like 5-fold validation.|
| **An Innovative Approach for PCO Morphology Segmentation Using MOT-SF**         | - Develops the MOT-SF algorithm, a multiscale gradient-weighted Otsu thresholding technique with fusion.      |
|                                                                                 | - Improves segmentation of small follicles in noisy, low-resolution ultrasound images.                        |
|                                                                                 | - Reports high precision (94%) and accuracy (85%) compared to traditional methods.                            |



<h4 style='color:brown'> 1. An extended machine learning technique for polycystic ovary syndrome detection using ovary ultrasound image </h4>

 
<span style='color:green'> _**Hyperparameters:**_

1. PCA Components: Tunable via n_components.
2. Gradient Boosting: n_estimators=100, learning_rate=0.1, max_depth=6.

### Image Processing

- Convert ultrasound images to grayscale.
- Apply histogram equalization for contrast enhancement.
- Segment ovarian regions using thresholding.

In [1]:
import cv2
import numpy as np
from skimage import exposure

def preprocess_image(image_path):
    """
    Preprocess the ultrasound image:
    - Convert to grayscale
    - Apply histogram equalization
    - Apply thresholding for segmentation
    """
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply histogram equalization
    equalized_img = exposure.equalize_hist(img)

    # Thresholding for segmentation
    _, segmented_img = cv2.threshold((equalized_img * 255).astype(np.uint8), 127, 255, cv2.THRESH_BINARY)

    return segmented_img


### Feature Extraction
- Extract handcrafted features like texture, shape, and statistical properties.
- Compute features from GLCM (Gray-Level Co-occurrence Matrix).

In [3]:
from skimage.feature import graycomatrix, graycoprops

def extract_features(image):
    """
    Extract features from the image:
    - Shape features: Area, Perimeter, Circularity
    - Texture features: GLCM properties
    - Statistical features: Mean, Variance, Skewness, Kurtosis
    """
    features = {}

    # Statistical Features
    features["mean"] = np.mean(image)
    features["variance"] = np.var(image)
    features["skewness"] = np.mean((image - features["mean"])**3) / (np.std(image)**3 + 1e-8)
    features["kurtosis"] = np.mean((image - features["mean"])**4) / (np.std(image)**4 + 1e-8)

    # Shape Features
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        perimeter = cv2.arcLength(cnt, True)
        area = cv2.contourArea(cnt)
        circularity = (4 * np.pi * area) / (perimeter**2 + 1e-8)
        features["area"] = area
        features["perimeter"] = perimeter
        features["circularity"] = circularity

    # Texture Features
    glcm = graycomatrix(image, distances=[1], angles=[0], symmetric=True, normed=True)
    features["contrast"] = graycoprops(glcm, "contrast")[0, 0]
    features["energy"] = graycoprops(glcm, "energy")[0, 0]
    features["homogeneity"] = graycoprops(glcm, "homogeneity")[0, 0]
    features["correlation"] = graycoprops(glcm, "correlation")[0, 0]

    return features


### Feature Selection [PCA]
-Reduce dimensionality using statistical methods

In [5]:
from sklearn.decomposition import PCA

def select_features(features, n_components=10):
    """
    Apply PCA to reduce feature dimensionality.
    """
    feature_array = np.array(features)
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(feature_array)
    return reduced_features


### Classification
- Train XGBoost classifiers on the extracted features for PCOS diagnosis.

### Install_XGBoost_Library

In [42]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/124.9 MB 3.6 MB/s eta 0:00:35
   ---------------------------------------- 0.3/124.9 MB 3.3 MB/s eta 0:00:38
   ---------------------------------------- 0.5/124.9 MB 3.9 MB/s eta 0:00:32
   ---------------------------------------- 0.7/124.9 MB 3.8 MB/s eta 0:00:33
   ---------------------------------------- 0.9/124.9 MB 3.9 MB/s eta 0:00:32
   ---------------------------------------- 1.2/124.9 MB 4.2 MB/s eta 0:00:30
   ---------------------------------------- 1.5/124.9 MB 4.5 MB/s eta 0:00:28
    --------------------------------------- 1.8/124.9 MB 4.9 MB/s eta 0:00:25
    --------------------------------------- 1.9/124.9 MB 4.9 MB/s eta 0:00:26
    --------------------------------------- 1.9/124.9 MB 4.9 MB/s eta 0:00:26
    --------------------------------------- 2.2/124.9 MB 4.4 MB/s eta 0:00:28
    --------------------------------------- 2.3/124.9 MB 4.1 MB/s eta 0

Collecting xgboost<br>
  Downloading xgboost-2.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)<br>
Requirement already satisfied: numpy in c:\users\soumiz\anaconda3\lib\site-packages (from xgboost) (1.26.4)<br>
Requirement already satisfied: scipy in c:\users\soumiz\anaconda3\lib\site-packages (from xgboost) (1.13.1)<br>
Downloading xgboost-2.1.2-py3-none-win_amd64.whl (124.9 MB)<br>
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--<br>
   ---------------------------------------- 0.1/124.9 MB 3.6 MB/s eta 0:00:35<br>
   ---------------------------------------- 0.3/124.9 MB 3.3 MB/s eta 0:00:38<br>
   ---------------------------------------- 0.5/124.9 MB 3.9 MB/s eta 0:00:32<br>
   ---------------------------------------- 0.7/124.9 MB 3.8 MB/s eta 0:00:33<br>
   ---------------------------------------- 0.9/124.9 MB 3.9 MB/s eta 0:00:32<br>
   ---------------------------------------- 1.2/124.9 MB 4.2 MB/s eta 0:00:30<br>
   ---------------------------------------- 1.5/124.9 MB 4.5 MB/s eta 0:00:28<br>
    --------------------------------------- 1.8/124.9 MB 4.9 MB/s eta 0:00:25<br>
    --------------------------------------- 1.9/124.9 MB 4.9 MB/s eta 0:00:26<br>
    --------------------------------------- 1.9/124.9 MB 4.9 MB/s eta 0:00:26<br>
    --------------------------------------- 2.2/124.9 MB 4.4 MB/s eta 0:00:28<br>
    --------------------------------------- 2.3/124.9 MB 4.1 MB/s eta 0:00:31<br>
    --------------------------------------- 2.5/124.9 MB 4.1 MB/s eta 0:00:30<br>
    --------------------------------------- 2.7/124.9 MB 4.1 MB/s eta 0:00:31<br>
    --------------------------------------- 2.7/124.9 MB 4.1 MB/s eta 0:00:31<br>
    --------------------------------------- 2.8/124.9 MB 3.9 MB/s eta 0:00:32<br>
    --------------------------------------- 2.9/124.9 MB 3.7 MB/s eta 0:00:33<br>
    --------------------------------------- 3.0/124.9 MB 3.6 MB/s eta 0:00:35<br>
    --------------------------------------- 3.1/124.9 MB 3.5 MB/s eta 0:00:35<br>
   - -------------------------------------- 3.1/124.9 MB 3.3 MB/s eta 0:00:37<br>
   - -------------------------------------- 3.2/124.9 MB 3.3 MB/s eta 0:00:38<br>
   - -------------------------------------- 3.2/124.9 MB 3.3 MB/s eta 0:00:38<br>
   - -------------------------------------- 3.2/124.9 MB 3.3 MB/s eta 0:00:38<br>
   - -------------------------------------- 3.4/124.9 MB 3.0 MB/s eta 0:00:41<br>
   - -------------------------------------- 3.5/124.9 MB 2.9 MB/s eta 0:00:42<br>
   - -------------------------------------- 3.6/124.9 MB 2.9 MB/s eta 0:00:43<br>
   - -------------------------------------- 3.6/124.9 MB 2.9 MB/s eta 0:00:43<br>
   - -------------------------------------- 3.6/124.9 MB 2.7 MB/s eta 0:00:45<br>
   - -------------------------------------- 3.7/124.9 MB 2.8 MB/s eta 0:00:45<br>
   - -------------------------------------- 3.9/124.9 MB 2.7 MB/s eta 0:00:45<br>
   - -------------------------------------- 4.0/124.9 MB 2.7 MB/s eta 0:00:45<br>
   - -------------------------------------- 4.2/124.9 MB 2.8 MB/s eta 0:00:44<br>
   - -------------------------------------- 4.4/124.9 MB 2.8 MB/s eta 0:00:43<br>
   - -------------------------------------- 4.6/124.9 MB 2.9 MB/s eta 0:00:42<br>
   - -------------------------------------- 4.9/124.9 MB 2.9 MB/s eta 0:00:42<br>
   - -------------------------------------- 5.1/124.9 MB 3.0 MB/s eta 0:00:41<br>
   - -------------------------------------- 5.3/124.9 MB 3.0 MB/s eta 0:00:41<br>
   - -------------------------------------- 5.5/124.9 MB 3.0 MB/s eta 0:00:40<br>
   - -------------------------------------- 5.6/124.9 MB 3.0 MB/s eta 0:00:40<br>
   - -------------------------------------- 5.8/124.9 MB 3.0 MB/s eta 0:00:40<br>
   - -------------------------------------- 5.9/124.9 MB 3.0 MB/s eta 0:00:40<br>
   - -------------------------------------- 6.0/124.9 MB 3.0 MB/s eta 0:00:40<br>
   - -------------------------------------- 6.1/124.9 MB 3.0 MB/s eta 0:00:41<br>
   -- ------------------------------------- 6.3/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 6.5/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 6.6/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 6.8/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 6.9/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 7.0/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 7.2/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 7.3/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 7.5/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 7.7/124.9 MB 3.0 MB/s eta 0:00:39<br>
   -- ------------------------------------- 7.9/124.9 MB 3.0 MB/s eta 0:00:39<br>
   -- ------------------------------------- 8.0/124.9 MB 3.0 MB/s eta 0:00:39<br>
   -- ------------------------------------- 8.1/124.9 MB 3.0 MB/s eta 0:00:39<br>
   -- ------------------------------------- 8.3/124.9 MB 3.0 MB/s eta 0:00:39<br>
   -- ------------------------------------- 8.4/124.9 MB 3.0 MB/s eta 0:00:39<br>
   -- ------------------------------------- 8.5/124.9 MB 3.0 MB/s eta 0:00:39<br>
   -- ------------------------------------- 8.5/124.9 MB 2.9 MB/s eta 0:00:40<br>
   -- ------------------------------------- 8.6/124.9 MB 3.0 MB/s eta 0:00:40<br>
   -- ------------------------------------- 8.7/124.9 MB 2.9 MB/s eta 0:00:40<br>
   -- ------------------------------------- 8.8/124.9 MB 2.9 MB/s eta 0:00:40<br>
   -- ------------------------------------- 8.9/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.0/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.1/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.1/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.1/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.1/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.1/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.1/124.9 MB 2.9 MB/s eta 0:00:41<br>
   -- ------------------------------------- 9.1/124.9 MB 2.6 MB/s eta 0:00:44<br>
   -- ------------------------------------- 9.1/124.9 MB 2.6 MB/s eta 0:00:45<br>
   -- ------------------------------------- 9.1/124.9 MB 2.6 MB/s eta 0:00:45<br>
   -- ------------------------------------- 9.1/124.9 MB 2.5 MB/s eta 0:00:46<br>
   -- ------------------------------------- 9.2/124.9 MB 2.5 MB/s eta 0:00:46<br>
   -- ------------------------------------- 9.3/124.9 MB 2.5 MB/s eta 0:00:47<br>
   --- ------------------------------------ 9.4/124.9 MB 2.5 MB/s eta 0:00:47<br>
   --- ------------------------------------ 9.5/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 9.6/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 9.8/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 9.9/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 10.1/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 10.2/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 10.4/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 10.5/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 10.6/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 10.8/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 11.0/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 11.1/124.9 MB 2.5 MB/s eta 0:00:46<br>
   --- ------------------------------------ 11.2/124.9 MB 2.5 MB/s eta 0:00:47<br>
   --- ------------------------------------ 11.2/124.9 MB 2.4 MB/s eta 0:00:47<br>
   --- ------------------------------------ 11.3/124.9 MB 2.4 MB/s eta 0:00:48<br>
   --- ------------------------------------ 11.4/124.9 MB 2.4 MB/s eta 0:00:48<br>
   --- ------------------------------------ 11.4/124.9 MB 2.4 MB/s eta 0:00:48<br>
   --- ------------------------------------ 11.4/124.9 MB 2.3 MB/s eta 0:00:49<br>
   --- ------------------------------------ 11.6/124.9 MB 2.3 MB/s eta 0:00:49<br>
   --- ------------------------------------ 11.6/124.9 MB 2.3 MB/s eta 0:00:49<br>
   --- ------------------------------------ 11.7/124.9 MB 2.3 MB/s eta 0:00:50<br>
   --- ------------------------------------ 11.7/124.9 MB 2.3 MB/s eta 0:00:50<br>
   --- ------------------------------------ 11.8/124.9 MB 2.2 MB/s eta 0:00:51<br>
   --- ------------------------------------ 11.8/124.9 MB 2.2 MB/s eta 0:00:52<br>
   --- ------------------------------------ 11.9/124.9 MB 2.2 MB/s eta 0:00:52<br>
   --- ------------------------------------ 12.0/124.9 MB 2.2 MB/s eta 0:00:52<br>
   --- ------------------------------------ 12.0/124.9 MB 2.2 MB/s eta 0:00:53<br>
   --- ------------------------------------ 12.1/124.9 MB 2.2 MB/s eta 0:00:53<br>
   --- ------------------------------------ 12.2/124.9 MB 2.2 MB/s eta 0:00:52<br>
   --- ------------------------------------ 12.3/124.9 MB 2.2 MB/s eta 0:00:53<br>
   --- ------------------------------------ 12.5/124.9 MB 2.2 MB/s eta 0:00:53<br>
   ---- ----------------------------------- 12.6/124.9 MB 2.2 MB/s eta 0:00:52<br>
   ---- ----------------------------------- 12.7/124.9 MB 2.2 MB/s eta 0:00:53<br>
   ---- ----------------------------------- 12.8/124.9 MB 2.1 MB/s eta 0:00:53<br>
   ---- ----------------------------------- 12.9/124.9 MB 2.1 MB/s eta 0:00:53<br>
   ---- ----------------------------------- 13.0/124.9 MB 2.1 MB/s eta 0:00:53<br>
   ---- ----------------------------------- 13.2/124.9 MB 2.2 MB/s eta 0:00:52<br>
   ---- ----------------------------------- 13.3/124.9 MB 2.2 MB/s eta 0:00:52<br>
   ---- ----------------------------------- 13.5/124.9 MB 2.3 MB/s eta 0:00:50<br>
   ---- ----------------------------------- 13.6/124.9 MB 2.2 MB/s eta 0:00:50<br>
   ---- ----------------------------------- 13.6/124.9 MB 2.2 MB/s eta 0:00:50<br>
   ---- ----------------------------------- 13.8/124.9 MB 2.3 MB/s eta 0:00:50<br>
   ---- ----------------------------------- 14.0/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.1/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.2/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.3/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.3/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.3/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.3/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.3/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.3/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.3/124.9 MB 2.3 MB/s eta 0:00:49<br>
   ---- ----------------------------------- 14.4/124.9 MB 2.1 MB/s eta 0:00:52<br>
   ---- ----------------------------------- 14.4/124.9 MB 2.1 MB/s eta 0:00:53<br>
   ---- ----------------------------------- 14.4/124.9 MB 2.1 MB/s eta 0:00:53<br>
   ---- ----------------------------------- 14.5/124.9 MB 2.1 MB/s eta 0:00:54<br>
   ---- ----------------------------------- 14.6/124.9 MB 2.1 MB/s eta 0:00:54<br>
   ---- ----------------------------------- 14.7/124.9 MB 2.1 MB/s eta 0:00:54<br>
   ---- ----------------------------------- 14.8/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ---- ----------------------------------- 14.9/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ---- ----------------------------------- 15.0/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ---- ----------------------------------- 15.2/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ---- ----------------------------------- 15.3/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ---- ----------------------------------- 15.4/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ---- ----------------------------------- 15.5/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 15.7/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 15.8/124.9 MB 2.0 MB/s eta 0:00:56<br>
   ----- ---------------------------------- 15.9/124.9 MB 2.0 MB/s eta 0:00:56<br>
   ----- ---------------------------------- 16.0/124.9 MB 2.0 MB/s eta 0:00:56<br>
   ----- ---------------------------------- 16.1/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 16.3/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 16.4/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 16.6/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 16.7/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 16.9/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 17.0/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 17.2/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ----- ---------------------------------- 17.4/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 17.6/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 17.7/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.0/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.1/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.2/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.2/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.2/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.2/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.2/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.2/124.9 MB 2.0 MB/s eta 0:00:54<br>
   ----- ---------------------------------- 18.3/124.9 MB 1.9 MB/s eta 0:00:57<br>
   ----- ---------------------------------- 18.3/124.9 MB 1.9 MB/s eta 0:00:57<br>
   ----- ---------------------------------- 18.4/124.9 MB 1.9 MB/s eta 0:00:57<br>
   ----- ---------------------------------- 18.6/124.9 MB 1.9 MB/s eta 0:00:57<br>
   ------ --------------------------------- 18.9/124.9 MB 1.9 MB/s eta 0:00:55<br>
   ------ --------------------------------- 19.1/124.9 MB 2.0 MB/s eta 0:00:55<br>
   ------ --------------------------------- 19.4/124.9 MB 2.1 MB/s eta 0:00:50<br>
   ------ --------------------------------- 19.6/124.9 MB 2.2 MB/s eta 0:00:49<br>
   ------ --------------------------------- 19.8/124.9 MB 2.2 MB/s eta 0:00:48<br>
   ------ --------------------------------- 20.1/124.9 MB 2.2 MB/s eta 0:00:48<br>
   ------ --------------------------------- 20.3/124.9 MB 2.2 MB/s eta 0:00:47<br>
   ------ --------------------------------- 20.6/124.9 MB 2.3 MB/s eta 0:00:47<br>
   ------ --------------------------------- 20.9/124.9 MB 2.3 MB/s eta 0:00:46<br>
   ------ --------------------------------- 21.2/124.9 MB 2.3 MB/s eta 0:00:45<br>
   ------ --------------------------------- 21.4/124.9 MB 2.4 MB/s eta 0:00:44<br>
   ------ --------------------------------- 21.7/124.9 MB 2.5 MB/s eta 0:00:42<br>
   ------- -------------------------------- 21.9/124.9 MB 2.5 MB/s eta 0:00:41<br>
   ------- -------------------------------- 22.1/124.9 MB 2.6 MB/s eta 0:00:39<br>
   ------- -------------------------------- 22.4/124.9 MB 2.7 MB/s eta 0:00:39<br>
   ------- -------------------------------- 22.6/124.9 MB 2.7 MB/s eta 0:00:38<br>
   ------- -------------------------------- 22.8/124.9 MB 2.8 MB/s eta 0:00:38<br>
   ------- -------------------------------- 23.0/124.9 MB 2.8 MB/s eta 0:00:37<br>
   ------- -------------------------------- 23.1/124.9 MB 2.8 MB/s eta 0:00:37<br>
   ------- -------------------------------- 23.3/124.9 MB 2.8 MB/s eta 0:00:37<br>
   ------- -------------------------------- 23.6/124.9 MB 2.8 MB/s eta 0:00:36<br>
   ------- -------------------------------- 23.9/124.9 MB 2.9 MB/s eta 0:00:35<br>
   ------- -------------------------------- 24.0/124.9 MB 2.9 MB/s eta 0:00:35<br>
   ------- -------------------------------- 24.0/124.9 MB 2.9 MB/s eta 0:00:35<br>
   ------- -------------------------------- 24.0/124.9 MB 2.9 MB/s eta 0:00:35<br>
   ------- -------------------------------- 24.4/124.9 MB 2.9 MB/s eta 0:00:36<br>
   ------- -------------------------------- 24.7/124.9 MB 3.4 MB/s eta 0:00:30<br>
   ------- -------------------------------- 24.9/124.9 MB 3.4 MB/s eta 0:00:30<br>
   -------- ------------------------------- 25.0/124.9 MB 3.4 MB/s eta 0:00:30<br>
   -------- ------------------------------- 25.2/124.9 MB 3.5 MB/s eta 0:00:29<br>
   -------- ------------------------------- 25.4/124.9 MB 3.5 MB/s eta 0:00:29<br>
   -------- ------------------------------- 25.6/124.9 MB 3.5 MB/s eta 0:00:29<br>
   -------- ------------------------------- 25.8/124.9 MB 3.6 MB/s eta 0:00:28<br>
   -------- ------------------------------- 26.0/124.9 MB 3.6 MB/s eta 0:00:28<br>
   -------- ------------------------------- 26.1/124.9 MB 3.6 MB/s eta 0:00:28<br>
   -------- ------------------------------- 26.3/124.9 MB 3.7 MB/s eta 0:00:27<br>
   -------- ------------------------------- 26.5/124.9 MB 3.7 MB/s eta 0:00:27<br>
   -------- ------------------------------- 26.7/124.9 MB 3.7 MB/s eta 0:00:27<br>
   -------- ------------------------------- 26.9/124.9 MB 3.7 MB/s eta 0:00:27<br>
   -------- ------------------------------- 27.2/124.9 MB 3.7 MB/s eta 0:00:27<br>
   -------- ------------------------------- 27.4/124.9 MB 3.8 MB/s eta 0:00:26<br>
   -------- ------------------------------- 27.7/124.9 MB 3.8 MB/s eta 0:00:26<br>
   -------- ------------------------------- 28.0/124.9 MB 3.9 MB/s eta 0:00:26<br>
   --------- ------------------------------ 28.3/124.9 MB 3.9 MB/s eta 0:00:25<br>
   --------- ------------------------------ 28.5/124.9 MB 4.4 MB/s eta 0:00:23<br>
   --------- ------------------------------ 28.7/124.9 MB 4.5 MB/s eta 0:00:22<br>
   --------- ------------------------------ 28.9/124.9 MB 4.5 MB/s eta 0:00:22<br>
   --------- ------------------------------ 28.9/124.9 MB 4.5 MB/s eta 0:00:22<br>
   --------- ------------------------------ 29.1/124.9 MB 4.4 MB/s eta 0:00:22<br>
   --------- ------------------------------ 29.3/124.9 MB 4.4 MB/s eta 0:00:22<br>
   --------- ------------------------------ 29.5/124.9 MB 4.4 MB/s eta 0:00:22<br>
   --------- ------------------------------ 29.7/124.9 MB 4.4 MB/s eta 0:00:22<br>
   --------- ------------------------------ 29.9/124.9 MB 4.3 MB/s eta 0:00:22<br>
   --------- ------------------------------ 30.2/124.9 MB 4.4 MB/s eta 0:00:22<br>
   --------- ------------------------------ 30.5/124.9 MB 4.4 MB/s eta 0:00:22<br>
   --------- ------------------------------ 30.8/124.9 MB 4.4 MB/s eta 0:00:22<br>
   --------- ------------------------------ 31.1/124.9 MB 4.4 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 31.3/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 31.5/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 31.7/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 31.9/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 32.1/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 32.4/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 32.6/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 32.8/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ---------- ----------------------------- 32.9/124.9 MB 4.2 MB/s eta 0:00:23<br>
   ---------- ----------------------------- 33.1/124.9 MB 4.0 MB/s eta 0:00:23<br>
   ---------- ----------------------------- 33.5/124.9 MB 4.1 MB/s eta 0:00:23<br>
   ---------- ----------------------------- 33.8/124.9 MB 4.1 MB/s eta 0:00:23<br>
   ---------- ----------------------------- 34.0/124.9 MB 4.1 MB/s eta 0:00:23<br>
   ---------- ----------------------------- 34.3/124.9 MB 4.3 MB/s eta 0:00:22<br>
   ----------- ---------------------------- 34.5/124.9 MB 4.2 MB/s eta 0:00:22<br>
   ----------- ---------------------------- 34.8/124.9 MB 4.2 MB/s eta 0:00:22<br>
   ----------- ---------------------------- 35.1/124.9 MB 4.2 MB/s eta 0:00:22<br>
   ----------- ---------------------------- 35.3/124.9 MB 4.3 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 35.5/124.9 MB 4.3 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 35.7/124.9 MB 4.3 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 35.9/124.9 MB 4.3 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 36.2/124.9 MB 4.3 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 36.5/124.9 MB 4.4 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 36.7/124.9 MB 4.4 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 37.0/124.9 MB 4.4 MB/s eta 0:00:21<br>
   ----------- ---------------------------- 37.2/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 37.5/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 37.8/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 38.0/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 38.2/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 38.5/124.9 MB 4.4 MB/s eta 0:00:20<br>
   ------------ --------------------------- 38.8/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 39.0/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 39.2/124.9 MB 4.5 MB/s eta 0:00:20<br>
   ------------ --------------------------- 39.3/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------ --------------------------- 39.6/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------ --------------------------- 39.9/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------ --------------------------- 40.2/124.9 MB 4.7 MB/s eta 0:00:19<br>
   ------------ --------------------------- 40.4/124.9 MB 4.7 MB/s eta 0:00:18<br>
   ------------- -------------------------- 40.6/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------- -------------------------- 40.8/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------- -------------------------- 41.1/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------- -------------------------- 41.2/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------- -------------------------- 41.4/124.9 MB 4.6 MB/s eta 0:00:19<br>
   ------------- -------------------------- 41.6/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 41.6/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 41.8/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 42.1/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 42.3/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 42.6/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 42.8/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 43.0/124.9 MB 4.5 MB/s eta 0:00:19<br>
   ------------- -------------------------- 43.2/124.9 MB 4.7 MB/s eta 0:00:18<br>
   ------------- -------------------------- 43.4/124.9 MB 4.9 MB/s eta 0:00:17<br>
   ------------- -------------------------- 43.7/124.9 MB 4.7 MB/s eta 0:00:18<br>
   -------------- ------------------------- 43.9/124.9 MB 4.8 MB/s eta 0:00:17<br>
   -------------- ------------------------- 44.1/124.9 MB 4.8 MB/s eta 0:00:17<br>
   -------------- ------------------------- 44.3/124.9 MB 4.7 MB/s eta 0:00:17<br>
   -------------- ------------------------- 44.4/124.9 MB 4.7 MB/s eta 0:00:17<br>
   -------------- ------------------------- 44.6/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 44.8/124.9 MB 4.7 MB/s eta 0:00:18<br>
   -------------- ------------------------- 45.0/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 45.2/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 45.4/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 45.6/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 45.8/124.9 MB 4.5 MB/s eta 0:00:18<br>
   -------------- ------------------------- 45.9/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 46.3/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 46.4/124.9 MB 4.6 MB/s eta 0:00:18<br>
   -------------- ------------------------- 46.7/124.9 MB 4.6 MB/s eta 0:00:18<br>
   --------------- ------------------------ 46.9/124.9 MB 4.5 MB/s eta 0:00:18<br>
   --------------- ------------------------ 47.1/124.9 MB 4.5 MB/s eta 0:00:18<br>
   --------------- ------------------------ 47.2/124.9 MB 4.5 MB/s eta 0:00:18<br>
   --------------- ------------------------ 47.4/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 47.6/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 47.9/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 48.1/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 48.3/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 48.6/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 48.8/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 49.0/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 49.2/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 49.4/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 49.6/124.9 MB 4.4 MB/s eta 0:00:18<br>
   --------------- ------------------------ 49.8/124.9 MB 4.4 MB/s eta 0:00:18<br>
   ---------------- ----------------------- 50.1/124.9 MB 4.4 MB/s eta 0:00:18<br>
   ---------------- ----------------------- 50.3/124.9 MB 4.4 MB/s eta 0:00:18<br>
   ---------------- ----------------------- 50.6/124.9 MB 4.4 MB/s eta 0:00:18<br>
   ---------------- ----------------------- 50.8/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 51.0/124.9 MB 4.3 MB/s eta 0:00:18<br>
   ---------------- ----------------------- 51.2/124.9 MB 4.3 MB/s eta 0:00:18<br>
   ---------------- ----------------------- 51.4/124.9 MB 4.3 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 51.6/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 51.8/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 52.0/124.9 MB 4.5 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 52.2/124.9 MB 4.5 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 52.4/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 52.7/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ---------------- ----------------------- 53.0/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.1/124.9 MB 4.5 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.3/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.5/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.7/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.7/124.9 MB 4.4 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.9/124.9 MB 4.3 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.9/124.9 MB 4.3 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.9/124.9 MB 4.3 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 53.9/124.9 MB 4.3 MB/s eta 0:00:17<br>
   ----------------- ---------------------- 54.1/124.9 MB 4.0 MB/s eta 0:00:18<br>
   ----------------- ---------------------- 54.1/124.9 MB 4.0 MB/s eta 0:00:18<br>
   ----------------- ---------------------- 54.1/124.9 MB 3.9 MB/s eta 0:00:19<br>
   ----------------- ---------------------- 54.2/124.9 MB 3.9 MB/s eta 0:00:19<br>
   ----------------- ---------------------- 54.3/124.9 MB 3.8 MB/s eta 0:00:19<br>
   ----------------- ---------------------- 54.4/124.9 MB 3.8 MB/s eta 0:00:19<br>
   ----------------- ---------------------- 54.5/124.9 MB 3.8 MB/s eta 0:00:19<br>
   ----------------- ---------------------- 54.6/124.9 MB 3.7 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 54.7/124.9 MB 3.7 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 54.8/124.9 MB 3.7 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 54.9/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.0/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.6 MB/s eta 0:00:20<br>
   ----------------- ---------------------- 55.1/124.9 MB 3.1 MB/s eta 0:00:23<br>
   ----------------- ---------------------- 55.2/124.9 MB 3.0 MB/s eta 0:00:23<br>
   ----------------- ---------------------- 55.2/124.9 MB 3.0 MB/s eta 0:00:24<br>
   ----------------- ---------------------- 55.2/124.9 MB 3.0 MB/s eta 0:00:24<br>
   ----------------- ---------------------- 55.2/124.9 MB 3.0 MB/s eta 0:00:24<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.9 MB/s eta 0:00:25<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.9 MB/s eta 0:00:25<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.9 MB/s eta 0:00:25<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.9 MB/s eta 0:00:25<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.9 MB/s eta 0:00:25<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.9 MB/s eta 0:00:25<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.7 MB/s eta 0:00:27<br>
   ----------------- ---------------------- 55.2/124.9 MB 2.7 MB/s eta 0:00:27<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.6 MB/s eta 0:00:27<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.6 MB/s eta 0:00:27<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.6 MB/s eta 0:00:27<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.6 MB/s eta 0:00:27<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.5 MB/s eta 0:00:29<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.5 MB/s eta 0:00:29<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.5 MB/s eta 0:00:29<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.4 MB/s eta 0:00:29<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.4 MB/s eta 0:00:29<br>
   ----------------- ---------------------- 55.3/124.9 MB 2.4 MB/s eta 0:00:30<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.3 MB/s eta 0:00:30<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.3 MB/s eta 0:00:30<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.3 MB/s eta 0:00:30<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.3 MB/s eta 0:00:30<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.3 MB/s eta 0:00:30<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.3 MB/s eta 0:00:30<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.2 MB/s eta 0:00:32<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.2 MB/s eta 0:00:32<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.2 MB/s eta 0:00:33<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.2 MB/s eta 0:00:33<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.1 MB/s eta 0:00:33<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.1 MB/s eta 0:00:33<br>
   ----------------- ---------------------- 55.4/124.9 MB 2.1 MB/s eta 0:00:33<br>
   ----------------- ---------------------- 55.5/124.9 MB 2.1 MB/s eta 0:00:34<br>
   ----------------- ---------------------- 55.5/124.9 MB 2.0 MB/s eta 0:00:35<br>
   ----------------- ---------------------- 55.5/124.9 MB 2.0 MB/s eta 0:00:35<br>
   ----------------- ---------------------- 55.5/124.9 MB 2.0 MB/s eta 0:00:35<br>
   ----------------- ---------------------- 55.5/124.9 MB 2.0 MB/s eta 0:00:35<br>
   ----------------- ---------------------- 55.5/124.9 MB 2.0 MB/s eta 0:00:36<br>
   ----------------- ---------------------- 55.5/124.9 MB 2.0 MB/s eta 0:00:36<br>
   ----------------- ---------------------- 55.5/124.9 MB 1.9 MB/s eta 0:00:36<br>
   ----------------- ---------------------- 55.5/124.9 MB 1.9 MB/s eta 0:00:36<br>
   ----------------- ---------------------- 55.5/124.9 MB 1.9 MB/s eta 0:00:36<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.9 MB/s eta 0:00:37<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.9 MB/s eta 0:00:37<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.9 MB/s eta 0:00:38<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.9 MB/s eta 0:00:38<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.9 MB/s eta 0:00:38<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.9 MB/s eta 0:00:38<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.9 MB/s eta 0:00:38<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.8 MB/s eta 0:00:39<br>
   ----------------- ---------------------- 55.6/124.9 MB 1.8 MB/s eta 0:00:39<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.8 MB/s eta 0:00:40<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.8 MB/s eta 0:00:40<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.7 MB/s eta 0:00:40<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.7 MB/s eta 0:00:40<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.7 MB/s eta 0:00:40<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.7 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.7 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.7 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.6 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.6 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.6 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.6 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.6 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.7/124.9 MB 1.6 MB/s eta 0:00:42<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.6 MB/s eta 0:00:44<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.6 MB/s eta 0:00:44<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.6 MB/s eta 0:00:44<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:45<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:45<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:45<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:45<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:45<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:47<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:47<br>
   ----------------- ---------------------- 55.8/124.9 MB 1.5 MB/s eta 0:00:47<br>
   ----------------- ---------------------- 55.9/124.9 MB 1.5 MB/s eta 0:00:48<br>
   ----------------- ---------------------- 55.9/124.9 MB 1.5 MB/s eta 0:00:48<br>
   ----------------- ---------------------- 55.9/124.9 MB 1.5 MB/s eta 0:00:48<br>
   ----------------- ---------------------- 55.9/124.9 MB 1.5 MB/s eta 0:00:48<br>
   ----------------- ---------------------- 55.9/124.9 MB 1.4 MB/s eta 0:00:49<br>
   ----------------- ---------------------- 55.9/124.9 MB 1.4 MB/s eta 0:00:49<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.4 MB/s eta 0:00:50<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.4 MB/s eta 0:00:50<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.4 MB/s eta 0:00:50<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.4 MB/s eta 0:00:50<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.4 MB/s eta 0:00:50<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.4 MB/s eta 0:00:50<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.3 MB/s eta 0:00:52<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.3 MB/s eta 0:00:53<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.3 MB/s eta 0:00:53<br>
   ----------------- ---------------------- 56.0/124.9 MB 1.3 MB/s eta 0:00:53<br>
   ----------------- ---------------------- 56.1/124.9 MB 1.3 MB/s eta 0:00:54<br>
   ----------------- ---------------------- 56.1/124.9 MB 1.3 MB/s eta 0:00:54<br>
   ----------------- ---------------------- 56.1/124.9 MB 1.3 MB/s eta 0:00:54<br>
   ----------------- ---------------------- 56.1/124.9 MB 1.3 MB/s eta 0:00:54<br>
   ----------------- ---------------------- 56.1/124.9 MB 1.3 MB/s eta 0:00:54<br>
   ----------------- ---------------------- 56.1/124.9 MB 1.3 MB/s eta 0:00:55<br>
   ----------------- ---------------------- 56.1/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ----------------- ---------------------- 56.2/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 56.3/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 56.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 56.5/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 56.6/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 56.8/124.9 MB 1.2 MB/s eta 0:00:57<br>
   ------------------ --------------------- 56.9/124.9 MB 1.2 MB/s eta 0:00:57<br>
   ------------------ --------------------- 57.0/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 57.2/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 57.3/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 57.5/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 57.6/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 57.7/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 57.8/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 58.0/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 58.1/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 58.3/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 58.4/124.9 MB 1.2 MB/s eta 0:00:57<br>
   ------------------ --------------------- 58.5/124.9 MB 1.2 MB/s eta 0:00:57<br>
   ------------------ --------------------- 58.7/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 58.8/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 58.9/124.9 MB 1.2 MB/s eta 0:00:57<br>
   ------------------ --------------------- 59.1/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------ --------------------- 59.2/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 59.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 59.6/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 59.7/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 59.9/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.1/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.3/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.4/124.9 MB 1.2 MB/s eta 0:00:56<br>
   ------------------- -------------------- 60.5/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 60.5/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 60.6/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 60.7/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 60.9/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.2/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.4/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.5/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.5/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.7/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.8/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 61.9/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:00:58<br>
   ------------------- -------------------- 62.1/124.9 MB 1.1 MB/s eta 0:01:00<br>
   ------------------- -------------------- 62.2/124.9 MB 1.1 MB/s eta 0:01:00<br>
   ------------------- -------------------- 62.2/124.9 MB 1.0 MB/s eta 0:01:00<br>
   ------------------- -------------------- 62.2/124.9 MB 1.0 MB/s eta 0:01:00<br>
   ------------------- -------------------- 62.3/124.9 MB 1.0 MB/s eta 0:01:01<br>
   ------------------- -------------------- 62.4/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 62.5/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 62.6/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 62.7/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 62.9/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.0/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.1/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.2/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.3/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.4/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.6/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.7/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.7/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 63.8/124.9 MB 1.0 MB/s eta 0:01:02<br>
   ------------------- ------------------- 63.9/124.9 MB 999.6 kB/s eta 0:01:02<br>
   -------------------- ------------------- 64.0/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 64.1/124.9 MB 1.0 MB/s eta 0:01:01<br>
   -------------------- ------------------- 64.2/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 64.3/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 64.3/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 64.3/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 64.3/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 64.3/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 64.3/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------ 64.4/124.9 MB 998.0 kB/s eta 0:01:01<br>
   -------------------- ------------------ 64.4/124.9 MB 993.5 kB/s eta 0:01:01<br>
   -------------------- ------------------ 64.5/124.9 MB 990.4 kB/s eta 0:01:01<br>
   -------------------- ------------------ 64.7/124.9 MB 993.6 kB/s eta 0:01:01<br>
   -------------------- ------------------ 64.8/124.9 MB 995.0 kB/s eta 0:01:01<br>
   -------------------- ------------------ 64.9/124.9 MB 998.0 kB/s eta 0:01:01<br>
   -------------------- ------------------- 65.1/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 65.3/124.9 MB 1.0 MB/s eta 0:01:00<br>
   -------------------- ------------------- 65.4/124.9 MB 1.1 MB/s eta 0:00:54<br>
   --------------------- ------------------ 65.6/124.9 MB 1.2 MB/s eta 0:00:49<br>
   --------------------- ------------------ 65.8/124.9 MB 1.3 MB/s eta 0:00:45<br>
   --------------------- ------------------ 65.9/124.9 MB 1.5 MB/s eta 0:00:41<br>
   --------------------- ------------------ 66.1/124.9 MB 1.6 MB/s eta 0:00:36<br>
   --------------------- ------------------ 66.2/124.9 MB 1.9 MB/s eta 0:00:32<br>
   --------------------- ------------------ 66.3/124.9 MB 2.1 MB/s eta 0:00:29<br>
   --------------------- ------------------ 66.5/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 66.6/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 66.7/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 66.8/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.0/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.1/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.3/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.5/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.6/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.7/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 67.9/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.0/124.9 MB 2.1 MB/s eta 0:00:27<br>
   --------------------- ------------------ 68.1/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.1 MB/s eta 0:00:28<br>
   --------------------- ------------------ 68.2/124.9 MB 2.0 MB/s eta 0:00:29<br>
   --------------------- ------------------ 68.3/124.9 MB 2.0 MB/s eta 0:00:29<br>
   --------------------- ------------------ 68.3/124.9 MB 1.9 MB/s eta 0:00:30<br>
   --------------------- ------------------ 68.3/124.9 MB 1.9 MB/s eta 0:00:30<br>
   --------------------- ------------------ 68.5/124.9 MB 1.9 MB/s eta 0:00:30<br>
   --------------------- ------------------ 68.6/124.9 MB 1.9 MB/s eta 0:00:29<br>
   ---------------------- ----------------- 68.8/124.9 MB 1.9 MB/s eta 0:00:29<br>
   ---------------------- ----------------- 69.0/124.9 MB 1.9 MB/s eta 0:00:29<br>
   ---------------------- ----------------- 69.1/124.9 MB 1.9 MB/s eta 0:00:29<br>
   ---------------------- ----------------- 69.3/124.9 MB 1.9 MB/s eta 0:00:29<br>
   ---------------------- ----------------- 69.5/124.9 MB 2.0 MB/s eta 0:00:29<br>
   ---------------------- ----------------- 69.7/124.9 MB 2.0 MB/s eta 0:00:29<br>
   ---------------------- ----------------- 70.0/124.9 MB 2.0 MB/s eta 0:00:28<br>
   ---------------------- ----------------- 70.1/124.9 MB 2.0 MB/s eta 0:00:28<br>
   ---------------------- ----------------- 70.3/124.9 MB 2.0 MB/s eta 0:00:28<br>
   ---------------------- ----------------- 70.5/124.9 MB 2.0 MB/s eta 0:00:28<br>
   ---------------------- ----------------- 70.7/124.9 MB 2.1 MB/s eta 0:00:26<br>
   ---------------------- ----------------- 71.0/124.9 MB 2.1 MB/s eta 0:00:26<br>
   ---------------------- ----------------- 71.1/124.9 MB 2.1 MB/s eta 0:00:26<br>
   ---------------------- ----------------- 71.2/124.9 MB 2.1 MB/s eta 0:00:26<br>
   ---------------------- ----------------- 71.3/124.9 MB 2.1 MB/s eta 0:00:27<br>
   ---------------------- ----------------- 71.3/124.9 MB 2.1 MB/s eta 0:00:27<br>
   ---------------------- ----------------- 71.6/124.9 MB 2.0 MB/s eta 0:00:27<br>
   ---------------------- ----------------- 71.8/124.9 MB 2.0 MB/s eta 0:00:27<br>
   ----------------------- ---------------- 71.9/124.9 MB 2.0 MB/s eta 0:00:26<br>
   ----------------------- ---------------- 72.1/124.9 MB 2.1 MB/s eta 0:00:26<br>
   ----------------------- ---------------- 72.3/124.9 MB 2.1 MB/s eta 0:00:26<br>
   ----------------------- ---------------- 72.6/124.9 MB 2.3 MB/s eta 0:00:23<br>
   ----------------------- ---------------- 72.9/124.9 MB 2.4 MB/s eta 0:00:23<br>
   ----------------------- ---------------- 73.1/124.9 MB 2.4 MB/s eta 0:00:22<br>
   ----------------------- ---------------- 73.3/124.9 MB 2.4 MB/s eta 0:00:22<br>
   ----------------------- ---------------- 73.5/124.9 MB 2.4 MB/s eta 0:00:22<br>
   ----------------------- ---------------- 73.7/124.9 MB 2.5 MB/s eta 0:00:21<br>
   ----------------------- ---------------- 73.9/124.9 MB 2.5 MB/s eta 0:00:21<br>
   ----------------------- ---------------- 74.1/124.9 MB 2.5 MB/s eta 0:00:21<br>
   ----------------------- ---------------- 74.3/124.9 MB 2.5 MB/s eta 0:00:20<br>
   ----------------------- ---------------- 74.5/124.9 MB 2.6 MB/s eta 0:00:20<br>
   ----------------------- ---------------- 74.8/124.9 MB 2.8 MB/s eta 0:00:18<br>
   ----------------------- ---------------- 74.8/124.9 MB 2.8 MB/s eta 0:00:18<br>
   ------------------------ --------------- 75.2/124.9 MB 2.9 MB/s eta 0:00:18<br>
   ------------------------ --------------- 75.3/124.9 MB 2.9 MB/s eta 0:00:18<br>
   ------------------------ --------------- 75.7/124.9 MB 2.9 MB/s eta 0:00:18<br>
   ------------------------ --------------- 75.8/124.9 MB 2.9 MB/s eta 0:00:17<br>
   ------------------------ --------------- 76.1/124.9 MB 2.9 MB/s eta 0:00:17<br>
   ------------------------ --------------- 76.2/124.9 MB 2.9 MB/s eta 0:00:17<br>
   ------------------------ --------------- 76.4/124.9 MB 2.9 MB/s eta 0:00:17<br>
   ------------------------ --------------- 76.6/124.9 MB 2.9 MB/s eta 0:00:17<br>
   ------------------------ --------------- 76.8/124.9 MB 3.0 MB/s eta 0:00:17<br>
   ------------------------ --------------- 77.0/124.9 MB 3.0 MB/s eta 0:00:17<br>
   ------------------------ --------------- 77.2/124.9 MB 3.0 MB/s eta 0:00:16<br>
   ------------------------ --------------- 77.4/124.9 MB 3.1 MB/s eta 0:00:16<br>
   ------------------------ --------------- 77.6/124.9 MB 3.1 MB/s eta 0:00:16<br>
   ------------------------ --------------- 77.8/124.9 MB 3.1 MB/s eta 0:00:16<br>
   ------------------------ --------------- 78.0/124.9 MB 3.1 MB/s eta 0:00:16<br>
   ------------------------- -------------- 78.3/124.9 MB 3.2 MB/s eta 0:00:15<br>
   ------------------------- -------------- 78.5/124.9 MB 3.6 MB/s eta 0:00:13<br>
   ------------------------- -------------- 78.7/124.9 MB 3.8 MB/s eta 0:00:13<br>
   ------------------------- -------------- 78.9/124.9 MB 3.8 MB/s eta 0:00:13<br>
   ------------------------- -------------- 79.2/124.9 MB 3.8 MB/s eta 0:00:12<br>
   ------------------------- -------------- 79.4/124.9 MB 3.8 MB/s eta 0:00:12<br>
   ------------------------- -------------- 79.6/124.9 MB 3.9 MB/s eta 0:00:12<br>
   ------------------------- -------------- 79.8/124.9 MB 3.9 MB/s eta 0:00:12<br>
   ------------------------- -------------- 80.0/124.9 MB 3.8 MB/s eta 0:00:12<br>
   ------------------------- -------------- 80.1/124.9 MB 3.8 MB/s eta 0:00:12<br>
   ------------------------- -------------- 80.4/124.9 MB 3.8 MB/s eta 0:00:12<br>
   ------------------------- -------------- 80.6/124.9 MB 3.9 MB/s eta 0:00:12<br>
   ------------------------- -------------- 80.8/124.9 MB 3.9 MB/s eta 0:00:12<br>
   ------------------------- -------------- 81.0/124.9 MB 3.8 MB/s eta 0:00:12<br>
   -------------------------- ------------- 81.3/124.9 MB 3.8 MB/s eta 0:00:12<br>
   -------------------------- ------------- 81.5/124.9 MB 3.9 MB/s eta 0:00:12<br>
   -------------------------- ------------- 81.7/124.9 MB 4.3 MB/s eta 0:00:11<br>
   -------------------------- ------------- 81.9/124.9 MB 4.3 MB/s eta 0:00:11<br>
   -------------------------- ------------- 82.1/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 82.3/124.9 MB 4.4 MB/s eta 0:00:10<br>
   -------------------------- ------------- 82.5/124.9 MB 4.4 MB/s eta 0:00:10<br>
   -------------------------- ------------- 82.7/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 82.9/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 83.1/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 83.4/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 83.6/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 83.8/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 84.0/124.9 MB 4.3 MB/s eta 0:00:10<br>
   -------------------------- ------------- 84.2/124.9 MB 4.4 MB/s eta 0:00:10<br>
   -------------------------- ------------- 84.2/124.9 MB 4.3 MB/s eta 0:00:10<br>
   --------------------------- ------------ 84.4/124.9 MB 4.2 MB/s eta 0:00:10<br>
   --------------------------- ------------ 84.6/124.9 MB 4.2 MB/s eta 0:00:10<br>
   --------------------------- ------------ 84.8/124.9 MB 4.2 MB/s eta 0:00:10<br>
   --------------------------- ------------ 84.9/124.9 MB 4.2 MB/s eta 0:00:10<br>
   --------------------------- ------------ 84.9/124.9 MB 4.1 MB/s eta 0:00:10<br>
   --------------------------- ------------ 85.1/124.9 MB 4.1 MB/s eta 0:00:10<br>
   --------------------------- ------------ 85.4/124.9 MB 4.1 MB/s eta 0:00:10<br>
   --------------------------- ------------ 85.6/124.9 MB 4.1 MB/s eta 0:00:10<br>
   --------------------------- ------------ 85.8/124.9 MB 4.1 MB/s eta 0:00:10<br>
   --------------------------- ------------ 85.8/124.9 MB 4.1 MB/s eta 0:00:10<br>
   --------------------------- ------------ 86.0/124.9 MB 4.0 MB/s eta 0:00:10<br>
   --------------------------- ------------ 86.2/124.9 MB 4.0 MB/s eta 0:00:10<br>
   --------------------------- ------------ 86.4/124.9 MB 4.0 MB/s eta 0:00:10<br>
   --------------------------- ------------ 86.6/124.9 MB 4.0 MB/s eta 0:00:10<br>
   --------------------------- ------------ 86.8/124.9 MB 4.0 MB/s eta 0:00:10<br>
   --------------------------- ------------ 86.9/124.9 MB 4.0 MB/s eta 0:00:10<br>
   --------------------------- ------------ 87.1/124.9 MB 4.0 MB/s eta 0:00:10<br>
   --------------------------- ------------ 87.4/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 87.5/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 87.7/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 87.9/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 88.1/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 88.4/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 88.6/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 88.8/124.9 MB 4.0 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 89.0/124.9 MB 3.9 MB/s eta 0:00:10<br>
   ---------------------------- ----------- 89.3/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ---------------------------- ----------- 89.5/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ---------------------------- ----------- 89.7/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ---------------------------- ----------- 89.9/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ---------------------------- ----------- 90.0/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ---------------------------- ----------- 90.1/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ---------------------------- ----------- 90.4/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 90.6/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 90.8/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 91.0/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 91.2/124.9 MB 4.0 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 91.4/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 91.6/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 91.8/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 91.9/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.1/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.3/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.4/124.9 MB 3.9 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.5/124.9 MB 3.8 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.7/124.9 MB 3.8 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.8/124.9 MB 3.8 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.8/124.9 MB 3.8 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.9/124.9 MB 3.7 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 92.9/124.9 MB 3.7 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 93.1/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 93.3/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 93.4/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ----------------------------- ---------- 93.6/124.9 MB 3.5 MB/s eta 0:00:09<br>
   ------------------------------ --------- 93.8/124.9 MB 3.5 MB/s eta 0:00:09<br>
   ------------------------------ --------- 93.9/124.9 MB 3.5 MB/s eta 0:00:09<br>
   ------------------------------ --------- 94.1/124.9 MB 3.5 MB/s eta 0:00:09<br>
   ------------------------------ --------- 94.3/124.9 MB 3.5 MB/s eta 0:00:09<br>
   ------------------------------ --------- 94.5/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ------------------------------ --------- 94.6/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ------------------------------ --------- 94.8/124.9 MB 3.5 MB/s eta 0:00:09<br>
   ------------------------------ --------- 95.0/124.9 MB 3.5 MB/s eta 0:00:09<br>
   ------------------------------ --------- 95.2/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ------------------------------ --------- 95.3/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ------------------------------ --------- 95.5/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ------------------------------ --------- 95.7/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ------------------------------ --------- 95.9/124.9 MB 3.6 MB/s eta 0:00:09<br>
   ------------------------------ --------- 96.1/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------ --------- 96.2/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------ --------- 96.3/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------ --------- 96.5/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------ --------- 96.7/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------- -------- 96.8/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.0/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.1/124.9 MB 3.5 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.2/124.9 MB 3.6 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.4/124.9 MB 3.5 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.5/124.9 MB 3.5 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.6/124.9 MB 3.5 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.8/124.9 MB 3.5 MB/s eta 0:00:08<br>
   ------------------------------- -------- 97.9/124.9 MB 3.5 MB/s eta 0:00:08<br>
   ------------------------------- -------- 98.0/124.9 MB 3.4 MB/s eta 0:00:08<br>
   ------------------------------- -------- 98.0/124.9 MB 3.4 MB/s eta 0:00:08<br>
   ------------------------------- -------- 98.0/124.9 MB 3.4 MB/s eta 0:00:09<br>
   ------------------------------- -------- 98.2/124.9 MB 3.3 MB/s eta 0:00:09<br>
   ------------------------------- -------- 98.3/124.9 MB 3.3 MB/s eta 0:00:09<br>
   ------------------------------- -------- 98.4/124.9 MB 3.3 MB/s eta 0:00:09<br>
   ------------------------------- -------- 98.6/124.9 MB 3.3 MB/s eta 0:00:09<br>
   ------------------------------- -------- 98.8/124.9 MB 3.3 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.0/124.9 MB 3.2 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.1/124.9 MB 3.2 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.2/124.9 MB 3.2 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.3/124.9 MB 3.2 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.5/124.9 MB 3.1 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.5/124.9 MB 3.1 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.5/124.9 MB 3.1 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.5/124.9 MB 3.1 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.5/124.9 MB 3.1 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.5/124.9 MB 3.1 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.6/124.9 MB 2.9 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.6/124.9 MB 2.9 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.6/124.9 MB 2.8 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.8/124.9 MB 2.8 MB/s eta 0:00:09<br>
   ------------------------------- -------- 99.9/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 100.1/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 100.3/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 100.5/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 100.7/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 100.8/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 100.9/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 101.0/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 101.2/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 101.4/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 101.5/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 101.7/124.9 MB 2.7 MB/s eta 0:00:09<br>
   -------------------------------- ------- 101.8/124.9 MB 2.7 MB/s eta 0:00:09<br>
   -------------------------------- ------- 102.0/124.9 MB 2.7 MB/s eta 0:00:09<br>
   -------------------------------- ------- 102.2/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 102.4/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 102.6/124.9 MB 2.8 MB/s eta 0:00:09<br>
   -------------------------------- ------- 102.7/124.9 MB 2.8 MB/s eta 0:00:08<br>
   -------------------------------- ------- 102.9/124.9 MB 2.8 MB/s eta 0:00:08<br>
   --------------------------------- ------ 103.1/124.9 MB 2.8 MB/s eta 0:00:08<br>
   --------------------------------- ------ 103.3/124.9 MB 2.9 MB/s eta 0:00:08<br>
   --------------------------------- ------ 103.5/124.9 MB 2.9 MB/s eta 0:00:08<br>
   --------------------------------- ------ 103.7/124.9 MB 2.9 MB/s eta 0:00:08<br>
   --------------------------------- ------ 103.8/124.9 MB 2.9 MB/s eta 0:00:08<br>
   --------------------------------- ------ 104.0/124.9 MB 2.9 MB/s eta 0:00:08<br>
   --------------------------------- ------ 104.2/124.9 MB 2.9 MB/s eta 0:00:08<br>
   --------------------------------- ------ 104.4/124.9 MB 2.9 MB/s eta 0:00:08<br>
   --------------------------------- ------ 104.6/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 104.7/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 104.9/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 105.0/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 105.2/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 105.4/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 105.6/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 105.7/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 105.9/124.9 MB 2.9 MB/s eta 0:00:07<br>
   --------------------------------- ------ 106.1/124.9 MB 2.9 MB/s eta 0:00:07<br>
   ---------------------------------- ----- 106.3/124.9 MB 2.9 MB/s eta 0:00:07<br>
   ---------------------------------- ----- 106.4/124.9 MB 2.9 MB/s eta 0:00:07<br>
   ---------------------------------- ----- 106.6/124.9 MB 2.9 MB/s eta 0:00:07<br>
   ---------------------------------- ----- 106.8/124.9 MB 2.9 MB/s eta 0:00:07<br>
   ---------------------------------- ----- 107.0/124.9 MB 2.9 MB/s eta 0:00:07<br>
   ---------------------------------- ----- 107.2/124.9 MB 2.9 MB/s eta 0:00:07<br>
   ---------------------------------- ----- 107.4/124.9 MB 2.9 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 107.6/124.9 MB 3.0 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 107.7/124.9 MB 3.0 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 108.0/124.9 MB 3.0 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 108.1/124.9 MB 3.0 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 108.3/124.9 MB 3.1 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 108.5/124.9 MB 3.1 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 108.7/124.9 MB 3.2 MB/s eta 0:00:06<br>
   ---------------------------------- ----- 108.9/124.9 MB 3.2 MB/s eta 0:00:05<br>
   ---------------------------------- ----- 109.1/124.9 MB 3.2 MB/s eta 0:00:05<br>
   ----------------------------------- ---- 109.4/124.9 MB 3.2 MB/s eta 0:00:05<br>
   ----------------------------------- ---- 109.6/124.9 MB 3.3 MB/s eta 0:00:05<br>
   ----------------------------------- ---- 109.8/124.9 MB 3.6 MB/s eta 0:00:05<br>
   ----------------------------------- ---- 109.9/124.9 MB 3.7 MB/s eta 0:00:05<br>
   ----------------------------------- ---- 110.1/124.9 MB 3.7 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 110.3/124.9 MB 3.7 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 110.6/124.9 MB 3.8 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 110.7/124.9 MB 3.8 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 111.0/124.9 MB 3.7 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 111.1/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 111.4/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 111.5/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 111.8/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 112.0/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 112.2/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ----------------------------------- ---- 112.4/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ------------------------------------ --- 112.5/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ------------------------------------ --- 112.7/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ------------------------------------ --- 113.0/124.9 MB 3.9 MB/s eta 0:00:04<br>
   ------------------------------------ --- 113.2/124.9 MB 3.9 MB/s eta 0:00:03<br>
   ------------------------------------ --- 113.3/124.9 MB 3.9 MB/s eta 0:00:03<br>
   ------------------------------------ --- 113.7/124.9 MB 4.0 MB/s eta 0:00:03<br>
   ------------------------------------ --- 113.9/124.9 MB 4.0 MB/s eta 0:00:03<br>
   ------------------------------------ --- 114.2/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------ --- 114.3/124.9 MB 4.0 MB/s eta 0:00:03<br>
   ------------------------------------ --- 114.5/124.9 MB 4.0 MB/s eta 0:00:03<br>
   ------------------------------------ --- 114.7/124.9 MB 4.0 MB/s eta 0:00:03<br>
   ------------------------------------ --- 114.9/124.9 MB 4.0 MB/s eta 0:00:03<br>
   ------------------------------------ --- 115.1/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------ --- 115.4/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------ --- 115.5/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------- -- 115.7/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------- -- 115.8/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------- -- 116.0/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------- -- 116.2/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------- -- 116.4/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------- -- 116.6/124.9 MB 4.1 MB/s eta 0:00:03<br>
   ------------------------------------- -- 116.8/124.9 MB 4.1 MB/s eta 0:00:02<br>
   ------------------------------------- -- 116.9/124.9 MB 4.1 MB/s eta 0:00:02<br>
   ------------------------------------- -- 117.1/124.9 MB 4.1 MB/s eta 0:00:02<br>
   ------------------------------------- -- 117.3/124.9 MB 4.1 MB/s eta 0:00:02<br>
   ------------------------------------- -- 117.5/124.9 MB 4.1 MB/s eta 0:00:02<br>
   ------------------------------------- -- 117.7/124.9 MB 4.1 MB/s eta 0:00:02<br>
   ------------------------------------- -- 118.0/124.9 MB 4.1 MB/s eta 0:00:02<br>
   ------------------------------------- -- 118.2/124.9 MB 4.2 MB/s eta 0:00:02<br>
   ------------------------------------- -- 118.4/124.9 MB 4.2 MB/s eta 0:00:02<br>
   ------------------------------------- -- 118.6/124.9 MB 4.2 MB/s eta 0:00:02<br>
   -------------------------------------- - 118.8/124.9 MB 4.2 MB/s eta 0:00:02<br>
   -------------------------------------- - 119.1/124.9 MB 4.2 MB/s eta 0:00:02<br>
   -------------------------------------- - 119.3/124.9 MB 4.2 MB/s eta 0:00:02<br>
   -------------------------------------- - 119.5/124.9 MB 4.2 MB/s eta 0:00:02<br>
   -------------------------------------- - 119.6/124.9 MB 4.1 MB/s eta 0:00:02<br>
   -------------------------------------- - 119.7/124.9 MB 4.1 MB/s eta 0:00:02<br>
   -------------------------------------- - 120.0/124.9 MB 4.1 MB/s eta 0:00:02<br>
   -------------------------------------- - 120.2/124.9 MB 4.1 MB/s eta 0:00:02<br>
   -------------------------------------- - 120.4/124.9 MB 4.1 MB/s eta 0:00:02<br>
   -------------------------------------- - 120.5/124.9 MB 4.1 MB/s eta 0:00:02<br>
   -------------------------------------- - 120.7/124.9 MB 4.1 MB/s eta 0:00:02<br>
   -------------------------------------- - 120.9/124.9 MB 4.1 MB/s eta 0:00:01<br>
   -------------------------------------- - 120.9/124.9 MB 4.0 MB/s eta 0:00:01<br>
   -------------------------------------- - 121.1/124.9 MB 4.0 MB/s eta 0:00:01<br>
   -------------------------------------- - 121.3/124.9 MB 4.0 MB/s eta 0:00:01<br>
   -------------------------------------- - 121.4/124.9 MB 4.0 MB/s eta 0:00:01<br>
   -------------------------------------- - 121.6/124.9 MB 4.0 MB/s eta 0:00:01<br>
   ---------------------------------------  121.8/124.9 MB 4.0 MB/s eta 0:00:01<br>
   ---------------------------------------  121.9/124.9 MB 4.0 MB/s eta 0:00:01<br>
   ---------------------------------------  122.2/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  122.3/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  122.5/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  122.6/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  122.8/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  123.0/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  123.1/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  123.2/124.9 MB 3.9 MB/s eta 0:00:01<br>
   ---------------------------------------  123.4/124.9 MB 3.8 MB/s eta 0:00:01<br>
   ---------------------------------------  123.6/124.9 MB 3.8 MB/s eta 0:00:01<br>
   ---------------------------------------  123.7/124.9 MB 3.8 MB/s eta 0:00:01<br>
   ---------------------------------------  123.9/124.9 MB 3.8 MB/s eta 0:00:01<br>
   ---------------------------------------  124.0/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.2/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.3/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.5/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.7/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------  124.9/124.9 MB 3.7 MB/s eta 0:00:01<br>
   ---------------------------------------- 124.9/124.9 MB 3.0 MB/s eta 0:00:00<br>
Installing collected packages: xgboost<br>
Successfully installed xgboost-2.1.2<br>
Note: you may need to restart the kernel to use updated packages.<br>


- ### Features of XGBoost (Extreme Gradient Boosting):

- Comes from the xgboost library.
- Provides additional features like:
- Regularization (lambda, alpha).
- GPU acceleration.
- Built-in handling of missing values.
- Faster computation due to optimized implementations.

In [43]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

def train_and_evaluate(features, labels):
    """
    Train and evaluate a classifier using XGBoost.
    """
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Train classifier
    classifier = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, use_label_encoder=False)
    classifier.fit(X_train, y_train)

    # Evaluate
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))
    return classifier


### Main Progam

In [44]:
import os

# Preprocessing, feature extraction, PCA, and classifier functions must be implemented as shown previously.

def load_data_from_folders(root_folder):
    """
    Load images and labels from the given folder structure.
    Folder names are used as labels for the images.
    """
    features = []
    labels = []

    # Walk through the folder structure
    for dataset_split in ["train copy", "validation", "testing"]:
        dataset_path = os.path.join(root_folder, "Standardized images", "dataverse_files_54 files", dataset_split)
        for label in os.listdir(dataset_path):
            label_path = os.path.join(dataset_path, label)
            if os.path.isdir(label_path):  # Ensure it's a directory
                for image_file in os.listdir(label_path):
                    image_path = os.path.join(label_path, image_file)
                    if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                        try:
                            # Preprocess the image
                            preprocessed_image = preprocess_image(image_path)

                            # Extract features
                            feature_vector = extract_features(preprocessed_image)

                            # Append features and labels
                            features.append(list(feature_vector.values()))
                            labels.append(0 if label == "normal" else 1)  # Encode labels: 0 for normal, 1 for pco
                        except Exception as e:
                            print(f"Error processing {image_path}: {e}")

    return features, labels

# Define paths
root_folder = r"D:\USD Univ. San Diego MS AI\AAI-521\Project Data"

# Load data
print("Loading data...")
features, labels = load_data_from_folders(root_folder)

# Feature selection
print("Applying PCA for feature selection...")
reduced_features = select_features(features, n_components=10)

# Train and evaluate classifier
print("Training and evaluating the classifier...")
train_and_evaluate(reduced_features, labels)


Loading data...
Applying PCA for feature selection...
Training and evaluating the classifier...
Accuracy: 0.73
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.00      0.00      0.00         1

    accuracy                           0.73        11
   macro avg       0.44      0.40      0.42        11
weighted avg       0.81      0.73      0.77        11



C:\Users\Soumiz\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:14:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)


<h4 style='color:brown'> 2. CystNet An AI driven model for PCOS detection using multilevel thresholding of ultrasound images </h4>


### Image Preprocessing with Multilevel Thresholding

- Convert images to grayscale.
- Apply multilevel thresholding for ovarian region segmentation.

In [8]:
import cv2
import numpy as np

def preprocess_with_thresholding(image_path):
    """
    Preprocess the image using multilevel thresholding.
    - Convert to grayscale.
    - Apply Otsu's method for multilevel thresholding.
    """
    # Load image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply GaussianBlur to reduce noise
    blurred_img = cv2.GaussianBlur(img, (5, 5), 0)

    # Apply Otsu's method for multilevel thresholding
    _, thresh1 = cv2.threshold(blurred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, thresh2 = cv2.threshold(blurred_img, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Combine thresholds
    multilevel_thresh = cv2.bitwise_or(thresh1, thresh2)

    return multilevel_thresh


### Data Augmentation

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_images(train_folder):
    """
    Use Keras ImageDataGenerator to perform augmentation on training images.
    - Rotation, zoom, flip, etc.
    """
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
    )

    train_generator = datagen.flow_from_directory(
        train_folder,
        target_size=(224, 224),
        color_mode="grayscale",
        class_mode="binary",
        batch_size=32,
        shuffle=True,
    )
    return train_generator


### CystNet Model Architecture

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_cystnet():
    """
    Build the CystNet architecture.
    """
    model = Sequential()

    # Convolutional Block 1
    model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 1)))
    model.add(MaxPooling2D((2, 2)))

    # Convolutional Block 2
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))

    # Convolutional Block 3
    model.add(Conv2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))  # Binary classification

    # Compile the model
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


### Training and Evaluation

In [34]:
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def train_and_evaluate(model, train_generator, validation_folder):
    """
    Train and evaluate the CystNet model and generate a classification report.
    """
    # Create validation data generator
    validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
    validation_generator = validation_datagen.flow_from_directory(
        validation_folder,
        target_size=(224, 224),
        color_mode="grayscale",
        class_mode="binary",
        batch_size=32,
        shuffle=False,
    )

    # Callbacks
    checkpoint = ModelCheckpoint("cystnet_best_model.keras", monitor="val_accuracy", save_best_only=True)
    early_stopping = EarlyStopping(monitor="val_loss", patience=5)

    # Train the model
    model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=50,
        callbacks=[checkpoint, early_stopping],
    )

    # Evaluate on validation data
    val_loss, val_accuracy = model.evaluate(validation_generator)
    print(f"Validation Accuracy: {val_accuracy:.2f}")

    # Generate predictions for the validation set
    val_predictions = (model.predict(validation_generator) > 0.5).astype(int)

    # Create the classification report
    true_labels = validation_generator.classes  # True labels from the validation generator
    class_labels = list(validation_generator.class_indices.keys())  # Class names
    report = classification_report(true_labels, val_predictions, target_names=class_labels)
    
    print("\nClassification Report:")
    print(report)


### Main Program

In [35]:
# Folder paths
root_folder = r"D:\USD Univ. San Diego MS AI\AAI-521\Project Data"
train_folder = os.path.join(root_folder, "Standardized images", "dataverse_files_54 files", "train copy")
validation_folder = os.path.join(root_folder, "Standardized images", "dataverse_files_54 files", "validation")
test_folder = os.path.join(root_folder, "Standardized images", "dataverse_files_54 files", "testing")

# Augment training images
print("Augmenting training images...")
train_generator = augment_images(train_folder)

# Build CystNet model
print("Building CystNet model...")
cystnet_model = build_cystnet()

# Train and evaluate
print("Training and evaluating the model...")
train_and_evaluate(cystnet_model, train_generator, validation_folder)

# Test on unseen data
print("Testing the model on unseen data...")
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    color_mode="grayscale",
    class_mode="binary",
    batch_size=32,
    shuffle=False,
)

test_loss, test_accuracy = cystnet_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")


Augmenting training images...
Found 32 images belonging to 2 classes.
Building CystNet model...


C:\Users\Soumiz\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Training and evaluating the model...
Found 11 images belonging to 2 classes.
Epoch 1/50


C:\Users\Soumiz\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step - accuracy: 0.2500 - loss: 18.2392 - val_accuracy: 0.7273 - val_loss: 2.0045
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.7500 - loss: 440.9052 - val_accuracy: 0.7273 - val_loss: 1.0488
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7500 - loss: 227.0822 - val_accuracy: 0.7273 - val_loss: 0.5938
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7500 - loss: 69.8399 - val_accuracy: 0.2727 - val_loss: 0.7170
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6562 - loss: 8.2154 - val_accuracy: 0.2727 - val_loss: 0.7153
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.2500 - loss: 19.6070 - val_accuracy: 0.2727 - val_loss: 0.6947
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4688 - loss: 3.0324 - val_accuracy: 0.7273 - val_loss: 0.6836
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7500 - loss: 2.0939 - val_accuracy: 0.7273 - val_loss: 0.6851
1/1 ━━━━━━━━━━━━━━

C:\Users\Soumiz\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Soumiz\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Soumiz\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Soumiz\anaconda3\Lib\site-pa

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step - accuracy: 0.7273 - loss: 0.6848
Test Accuracy: 0.73



<h4 style='color:brown'> 3. An innovative approach for PCO morphology segmentation using a novel MOT-SF technique

### Preprocessing and Segmentation

- Convert images to grayscale.
- Apply Gaussian smoothing to reduce noise.
- Perform histogram equalization for contrast enhancement.
- Use adaptive thresholding with Gaussian-weighted method.
- Apply connected component analysis to segment the ovarian region.

In [17]:
import cv2
import numpy as np

def preprocess_and_segment(image_path):
    """
    Preprocess and segment the image using MOT-SF:
    - Convert to grayscale.
    - Apply Gaussian smoothing and histogram equalization.
    - Perform adaptive thresholding for segmentation.
    """
    # Load image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian smoothing
    smoothed_img = cv2.GaussianBlur(img, (5, 5), 0)

    # Apply histogram equalization
    equalized_img = cv2.equalizeHist(smoothed_img)

    # Adaptive thresholding
    segmented_img = cv2.adaptiveThreshold(
        equalized_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )

    return segmented_img


### Feature Extraction

In [30]:
from skimage.measure import regionprops, label, shannon_entropy

def extract_morphological_features(segmented_image):
    """
    Extract morphological and statistical features from the segmented image:
    - Area, Perimeter, Aspect Ratio, Circularity.
    - Mean intensity, Variance, Entropy.
    """
    features = {}
    
    # Label connected components
    labeled_img = label(segmented_image)
    regions = regionprops(labeled_img)

    if regions:
        # Assume the largest connected component is the ovary
        largest_region = max(regions, key=lambda r: r.area)

        # Morphological Features
        area = largest_region.area
        perimeter = largest_region.perimeter
        bounding_box = largest_region.bbox
        aspect_ratio = (bounding_box[2] - bounding_box[0]) / (bounding_box[3] - bounding_box[1] + 1e-8)
        circularity = (4 * np.pi * area) / (perimeter**2 + 1e-8)

        features["area"] = area
        features["perimeter"] = perimeter
        features["aspect_ratio"] = aspect_ratio
        features["circularity"] = circularity

        # Statistical Features
        mask = largest_region.filled_image
        features["mean_intensity"] = np.mean(segmented_image[mask])
        features["variance"] = np.var(segmented_image[mask])
        features["entropy"] = shannon_entropy(segmented_image[mask])

    return features


### Loading Data and Extracting Features

In [31]:
import os

def load_data_and_extract_features(root_folder):
    """
    Load images and extract features from the given dataset.
    Folder names serve as labels.
    """
    features = []
    labels = []

    # Traverse the folder structure
    for dataset_split in ["train copy", "validation", "testing"]:
        dataset_path = os.path.join(root_folder, "Standardized images", "dataverse_files_54 files", dataset_split)
        for label in os.listdir(dataset_path):
            label_path = os.path.join(dataset_path, label)
            if os.path.isdir(label_path):
                for image_file in os.listdir(label_path):
                    image_path = os.path.join(label_path, image_file)
                    if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                        try:
                            # Preprocess and segment image
                            segmented_img = preprocess_and_segment(image_path)

                            # Extract features
                            feature_vector = extract_morphological_features(segmented_img)

                            # Append features and label
                            features.append(list(feature_vector.values()))
                            labels.append(0 if label == "normal" else 1)  # Encode labels: 0 for normal, 1 for pco
                        except Exception as e:
                            print(f"Error processing {image_path}: {e}")

    return features, labels


### Classifier Training and Evaluation

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

def train_and_evaluate(features, labels):
    """
    Train and evaluate a Random Forest classifier on the extracted features.
    """
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Train a Random Forest classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Evaluate the classifier
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))

    return clf


### Main Program

In [33]:
# Define root folder path
root_folder = r"D:\USD Univ. San Diego MS AI\AAI-521\Project Data"

# Load data and extract features
print("Loading data and extracting features...")
features, labels = load_data_and_extract_features(root_folder)

# Train and evaluate the classifier
print("Training and evaluating the classifier...")
train_and_evaluate(features, labels)


Loading data and extracting features...
Error processing D:\USD Univ. San Diego MS AI\AAI-521\Project Data\Standardized images\dataverse_files_54 files\train copy\normal\normal_14.jpg: boolean index did not match indexed array along dimension 0; dimension is 224 but corresponding boolean dimension is 151
Error processing D:\USD Univ. San Diego MS AI\AAI-521\Project Data\Standardized images\dataverse_files_54 files\train copy\normal\normal_20.jpg: boolean index did not match indexed array along dimension 0; dimension is 224 but corresponding boolean dimension is 134
Error processing D:\USD Univ. San Diego MS AI\AAI-521\Project Data\Standardized images\dataverse_files_54 files\train copy\normal\normal_27.jpg: boolean index did not match indexed array along dimension 0; dimension is 224 but corresponding boolean dimension is 149
Error processing D:\USD Univ. San Diego MS AI\AAI-521\Project Data\Standardized images\dataverse_files_54 files\train copy\normal\normal_29.jpg: boolean index did

C:\Users\Soumiz\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Soumiz\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Soumiz\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForestClassifier(random_state=42)

### Comparison Table for PCOS Detection Methods

| **Aspect**              | **An Extended ML Technique** | **CystNet**                                   | **MOT-SF**                                   |
|-------------------------|------------------------------|-----------------------------------------------|----------------------------------------------|
| **Segmentation**        | Focused on ensemble ML classifiers; no explicit segmentation | Uses CNN for segmentation                     | Novel multi-thresholding segmentation         |
| **Feature Extraction**  | PCA and morphological features | Automated via CNN layers                      | Morphological descriptors                    |
| **Classification**      | Gradient Boosting Classifier | End-to-end deep learning                      | ML on morphological features                 |
| **Dataset Handling**    | Labeled folders (normal/pco) | Augmentation and preprocessing applied        | Preprocessing with thresholding              |
| **Accuracy**            | **73%**               | **73%**      | **75%**                                      |

The performance metrics and implementation details provide distinct contributions:  
1. The **Extended ML Technique** achieves robust feature selection but lacks segmentation.  
2. **CystNet** excels in accuracy due to its deep learning architecture but is computationally heavy.  
3. **MOT-SF** introduces innovative segmentation.  
